In [ ]:
import pandas as pd
from etna.datasets import TSDataset

# Read the data
#df = pd.read_csv("examples/data/example_dataset.csv")
df = pd.read_csv("example_dataset.csv")

In [ ]:
!python -V

In [ ]:
df

In [ ]:
# Create a TSDataset
df = TSDataset.to_dataset(df)

In [ ]:
df

In [ ]:
type(df)

In [ ]:
df.columns

In [ ]:
ts = TSDataset(df, freq="D")

In [ ]:
ts

In [ ]:
type(ts)

In [ ]:
# Choose a horizon
HORIZON = 14

# Make train/test split
train_ts, test_ts = ts.train_test_split(test_size=HORIZON)

In [ ]:
train_ts

In [ ]:
test_ts

In [ ]:
from etna.models import LinearMultiSegmentModel

In [ ]:
from etna.models import NaiveModel

In [ ]:
from etna.models import CatBoostMultiSegmentModel
from etna.transforms import DateFlagsTransform
from etna.transforms import DensityOutliersTransform
from etna.transforms import FourierTransform
from etna.transforms import LagTransform
from etna.transforms import LinearTrendTransform
from etna.transforms import MeanTransform
from etna.transforms import SegmentEncoderTransform
from etna.transforms import TimeSeriesImputerTransform
from etna.transforms import TrendTransform

In [ ]:
# Prepare transforms
transforms = [
    DensityOutliersTransform(in_column="target", distance_coef=3.0),
    TimeSeriesImputerTransform(in_column="target", strategy="forward_fill"),
    LinearTrendTransform(in_column="target"),
    TrendTransform(in_column="target", out_column="trend"),
    LagTransform(in_column="target", lags=list(range(HORIZON, 122)), out_column="target_lag"),
    DateFlagsTransform(week_number_in_month=True, out_column="date_flag"),
    FourierTransform(period=360.25, order=6, out_column="fourier"),
    SegmentEncoderTransform(),
    MeanTransform(in_column=f"target_lag_{HORIZON}", window=12, seasonality=7),
    MeanTransform(in_column=f"target_lag_{HORIZON}", window=7),
]

In [ ]:
# Prepare model
#model = CatBoostMultiSegmentModel()
#model = LinearMultiSegmentModel()
model = NaiveModel(lag=1)

In [ ]:
%%time
from etna.pipeline import Pipeline

# Create and fit the pipeline
pipeline = Pipeline(model=model, transforms=transforms, horizon=HORIZON)
pipeline.fit(train_ts)

# Make a forecast
forecast_ts = pipeline.forecast()

In [ ]:
from etna.analysis import plot_forecast

plot_forecast(forecast_ts=forecast_ts, test_ts=test_ts, train_ts=train_ts, n_train_samples=50)

In [ ]:
from etna.metrics import SMAPE

metric = SMAPE(mode="macro")
metric_value = metric(y_true=test_ts, y_pred=forecast_ts)

In [ ]:
print(metric)

In [ ]:
print(metric_value)

In [ ]:
#4.59480745528492
#42.2346166371593